# Algoritmo para el entrenamiento de una red neuronal utilizando el descenso del gradiente

In [45]:
import numpy as np
import random

Inicialmente se procede a definir la función de activación y la derivada de la función de activación, para esto se utiliza la función sigmoide


\begin{equation}
g(z) = \frac{1}{1+e^{(-z)}}
\end{equation}

\begin{equation}
g'(z) = g(z)\cdot(1-g(z))
\end{equation}

In [46]:
def sigmoid(z):
    sig = 1.0/(1.0+np.exp(-z))
    return sig

In [47]:
def sigmoid_prime(z):
    sig_prime = sigmoid(z)*(1-sigmoid(z))
    return sig_prime

Se genera una clase para generar una red neuronal y entrenarla, esta clase toma el parámetro *sizes*, del tipo lista [numCapa1, numCapa2, numCapa3], donde el número de elementos de la lista representa el número de capas de la red y el valor de cada elemento el número de neuronas de cada capa, así mismo, [24,20,5], la capa1 constará de de 24 neuronas, la capa2 constará de 20 neuronas y la capa3 constará de 5 neuronas.

In [63]:
class Network(object):
    def __init__(self, sizes):
        # Sizes es una lista con el número de neuronas de cada capa
        self.num_layers = len(sizes)
        self.sizes = sizes
        # Se inicializan los bias de cada neurona con valores aleatorios
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # Se inicializan los pesos sinápticos de las conexiones con valores aleatorios
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        # Propagación hacia adelante, se calcula la activacion utilizando la función sigmoide
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w,a) + b)
        return a
    
    def SGD(self, train_data, epochs, mini_batch_size, eta, test_data = None):
        # Algoritmo para el descenso del gradiente
        training_data = list(train_data)
        n = len(training_data)
        
        if test_data:
            # Si se ingresan valores para el testeo se convierten a lista y se halla la dimensión
            test_data = list(test_data)
            n_test = len(test_data)
            
        for j in range(epochs):
            # Se "barajan" los datos de entrenamiento para seleccionar conjuntos aleatorios de datos
            random.shuffle(training_data)
            
            # Se crean lotes aleatorios del tamaño "mini_batch_size" 
            # La creación de lotes disminuye el costo computacional de la operación
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {}% - {}/{}". format(j, self.evaluate(test_data)*100/n_test, self.evaluate(test_data), n_test))
            else:
                print("Epocj {} complete".format(j))
    
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        # Se inicializan las matrices de bias y pesos sinápticos
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Se parte de la activación inicial, es decir el array X de entradas
        activation = x
        
        # Lista para almacenar las activaciones capa tras capa, se inicializa con el valor de las entradas X
        activations = [x]
        
        # Lista para almacenar los valores de z capa tras capa
        zs =[]
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_b, nabla_w
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x,y) in test_data]
        return sum(int(x == y) for (x,y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return output_activations-y
                      
            

In [64]:
import mnist_loader

In [65]:
Training_data, Validation_data, Test_data = mnist_loader.load_data_wrapper()

In [66]:
net = Network([784,30,10])

In [67]:
net.SGD(Training_data, 10, 10, 3.0, test_data = Test_data)

Epoch 0 : 90.97% - 9097/10000
Epoch 1 : 92.42% - 9242/10000
Epoch 2 : 93.07% - 9307/10000
Epoch 3 : 93.74% - 9374/10000
Epoch 4 : 93.89% - 9389/10000
Epoch 5 : 94.14% - 9414/10000
Epoch 6 : 94.38% - 9438/10000
Epoch 7 : 94.57% - 9457/10000
Epoch 8 : 94.7% - 9470/10000
Epoch 9 : 94.91% - 9491/10000
